

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/CLASSIFICATION_DE_NEWS.ipynb)




# **Classify German News**

## 1. Colab Setup

In [ ]:
# Install PySpark and Spark NLP
! pip install -q pyspark==3.3.0 spark-nlp==4.2.1

In [ ]:
import pandas as pd
import numpy as np
import json
from pyspark.ml import Pipeline
from pyspark.sql.types import StringType, IntegerType
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

## 2. Start Spark Session

In [ ]:
spark = sparknlp.start()

## 3. Select the DL model and re-run all the cells below

In [ ]:
model_name = "classifierdl_bert_news"

## 4. Some sample examples

In [ ]:
text_list = [
"""Dramatische Partie wird erst im Shootout entschieden bedeutet den ersten meistertitel seit 22 jahren. Wien a hc wien hat am sonntag beim final-vier-turnier der asterreichischen hockey-bundesliga triumphiert. Hc besiegte titelverteidiger sv arminen in einem dramatischen Endspiel 2:0 nach Shootout. Nach der verlangerung war es in wien 1:1 gestanden. """,
"""Vertrag bis saisonende a meistertitel soll her. Warschau a stanislaw tschertschessow ist neuer trainer von legia warschau. Der ehemalige tormann und trainer des fc tirol unterzeichnete beim polnischen cupsieger einen vertrag bis saisonende und tritt die nachfolge des entlassenen norwegischen ex-internationalen henning berg an.""",
"""Auch in den supermarkten machte sich die bankenkrise bemerkbar. Vor allem grundnahrungsmittel wie reis, mehl und bohnen wurden wieder in groayen mengen gekauft. Regale mit keksen und griechischem stangerlgeback waren weitgehend leer. Kunden schleppten groaye fanf-liter-kanister mit olivenal zur kasse. Dort wird weiter eher mit bargeld als mit kreditkarte bezahlt.""",
"""Rund 64.000 euro flossen von andra rupprechters ressort in die zeitschrift, an der der vp-bauernbund beteiligt ist. Wien a inserate der ministerien haben schon oft die gemater erregt. Sogar die justiz nahm sich der angelegenheit an, die staatsanwaltschaft stellte die ermittlungen wegen verdachts der untreue aber 2013 ein.""",
"""Vorstandsvorsitzender des fc bayern lasst interessenten abblitzen und will weitere starspieler nach manchen lotsen. Manchen a bayern manchens vorstandsvorsitzender karl-heinz rummenigge hat allen interessenten an fuayball-nationalspieler thomas maller erneut eine klare absage erteilt. Thomas ist unverkauflich, sagte er in einem interview des magazins sport bild (mittwoch).""",
"""Mit timothy peake geht erstmals auch ein britischer astronaut an bord der iss. Baikonur/london a am frahen dienstagabend erreichte eine sojus mit drei raumfahrern an bord die international space station (iss). Um 18:33 mez dockte das raumschiff rund 400 kilometer aber indien an, wie die nasa per kurznachrichtendienst twitter mitteilte. """,
"""Die beiden neuen modelle aberraschen mit niedrigen preisen, die vorab ins netz gelangt sind. Am dienstagabend wird google seine zwei neuen nexus-modelle offiziell vorstellen. Allerdings darfte es nur mehr wenige details geben, die interessierte beobachter aberraschen. """,
"""Ist es ein wal, der seeelefant oder das flusspferd? Der wettbewerb um das fetteste tier der erde hat einen verblaffenden sieger. London/wien a wahrend wir menschen an winterspeck wenig freude haben, entscheidet er im tierreich aber tod oder aberleben. Aber welches tier bringt davon nun am meisten auf die waage? Dieser frage, die gar nicht so leicht zu beantworten ist, ging die bbc karzlich auf die spur.""",
"""Youtuber nimmt apple-smartphones mit ak-74 unter beschuss. Aber langere zeit erfreuten sich sogenannte droptests auf youtube hoher beliebtheit. Auch heute noch lassen videomacher neue smartphones und ahnliche produkte, teilweise spektakular per zeitlupenkamera inszeniert, auf den boden fallen und versuchen, aus dem ergebnis auf die verarbeitungs- und materialqualitat des jeweiligen gerats zu schlieayen. """
]

## 5. Define Spark NLP pipeline

In [ ]:
document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

embeddings = BertSentenceEmbeddings\
  .pretrained('sent_bert_multi_cased', 'xx') \
  .setInputCols(["document"])\
  .setOutputCol("sentence_embeddings")

document_classifier = ClassifierDLModel.pretrained(model_name, "de") \
  .setInputCols(["document", "sentence_embeddings"]) \
  .setOutputCol("class_")  
     
nlpPipeline = Pipeline(
    stages=[
      document, 
      embeddings,
      document_classifier
      ])

sent_bert_multi_cased download started this may take some time.
Approximate size to download 638.6 MB
[OK!]
classifierdl_bert_news download started this may take some time.
Approximate size to download 21.6 MB
[OK!]


## 6. Run the pipeline

In [ ]:
df = spark.createDataFrame(text_list, StringType()).toDF("text")
result = nlpPipeline.fit(df).transform(df)

## 7. Visualize results

In [ ]:
result.select(F.explode(F.arrays_zip(result.document.result, 
                                     result.class_.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("document"),
              F.expr("cols['1']").alias("class")).show(truncate=150)

+------------------------------------------------------------------------------------------------------------------------------------------------------+------------+
|                                                                                                                                              document|       class|
+------------------------------------------------------------------------------------------------------------------------------------------------------+------------+
|Dramatische Partie wird erst im Shootout entschieden bedeutet den ersten meistertitel seit 22 jahren. Wien a hc wien hat am sonntag beim final-vier...|       Sport|
|Vertrag bis saisonende a meistertitel soll her. Warschau a stanislaw tschertschessow ist neuer trainer von legia warschau. Der ehemalige tormann un...|       Sport|
|Auch in den supermarkten machte sich die bankenkrise bemerkbar. Vor allem grundnahrungsmittel wie reis, mehl und bohnen wurden wieder in groayen me...|  Wirtschaft|
|Run